In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms
from torchvision import models

import os
import argparse
import time
from time import perf_counter
import sys

from matplotlib import pyplot as plt
import numpy as np

In [18]:
learning_rate = 0.001
num_epochs = 2
batch_size = 512
loss_function = torch.nn.CrossEntropyLoss()
dir_name = os.getcwd()

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

device(type='cuda', index=0)

## ResNet18

In [24]:
def current_accuracy(outputs, labels):
    _, predicted = torch.max(outputs.data, 1)
    return torch.tensor(torch.sum(predicted == labels).item() / len(predicted))

In [26]:
def train(model, train_data_loader, device, optimizer):
  for epoch in range(num_epochs):
    loss_list = []
    acc_list = []
    time_one = perf_counter()
    for batch_idx, (images, labels) in enumerate(train_data_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_function(outputs, labels)
        loss_list.append(loss)
        acc_list.append(current_accuracy(outputs, labels))
        loss.backward()
        optimizer.step()

    time_two = perf_counter()
    time = time_two-time_one
    print('Epoch {}: loss = {}, acc = {}, time = {}'.format(epoch, torch.stack(loss_list).mean(), torch.stack(acc_list).mean(), time))

In [9]:
train_dataset_resnet = torchvision.datasets.CIFAR10(
      root = dir_name, train = True, download = True,
      transform = models.ResNet18_Weights.DEFAULT.transforms()
)
test_dataset_resnet = torchvision.datasets.CIFAR10(
      root = dir_name, train = False, download = True,
      transform = models.ResNet18_Weights.DEFAULT.transforms()
)

100%|██████████| 170498071/170498071 [00:03<00:00, 48473566.93it/s]


Extracting /content/cifar-10-python.tar.gz to /content
Files already downloaded and verified


In [10]:
train_data_loader = torch.utils.data.DataLoader(
      train_dataset_resnet, batch_size = batch_size, shuffle = True
)
test_data_loader = torch.utils.data.DataLoader(
      test_dataset_resnet, batch_size = batch_size, shuffle = False
)

Плюс линейный слой

In [19]:
model = models.resnet18(models.ResNet18_Weights.DEFAULT)
for params in model.parameters():
  params.requires_grad = False

input_layers = model.fc.in_features
model.fc = torch.nn.Linear(input_layers, 10)
model.to(device)
optimizer = torch.optim.Adam(model.fc.parameters(), lr = learning_rate)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


Train

In [20]:
for epoch in range(num_epochs):
    loss_list = []
    acc_list = []
    time_one = perf_counter()
    for batch_idx, (images, labels) in enumerate(train_data_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_function(outputs, labels)
        loss_list.append(loss)
        acc_list.append(current_accuracy(outputs, labels))
        loss.backward()
        optimizer.step()

    time_two = perf_counter()
    time = time_two-time_one
    print('Epoch {}: loss = {}, acc = {}, time = {}'.format(epoch, torch.stack(loss_list).mean(), torch.stack(acc_list).mean(), time))

Epoch 0: loss = 1.3542511463165283, acc = 0.5782977342605591, time = 180.19985409599985
Epoch 1: loss = 0.8182274699211121, acc = 0.738261342048645, time = 174.757966141


Test

In [21]:
loss_list_test = []
acc_list_test = []
for batch_idx, (images, labels) in enumerate(test_data_loader):
    images, labels = images.to(device), labels.to(device)
    optimizer.zero_grad()
    outputs = model(images)
    loss = loss_function(outputs, labels)
    loss_list.append(loss)
    acc_list.append(current_accuracy(outputs, labels))
    loss.backward()
    optimizer.step()

print('Test loss = {}, Test acc = {}'.format(torch.stack(loss_list).mean(), torch.stack(acc_list).mean()))

Test loss = 0.8096103072166443, Test acc = 0.7398342490196228


Плюс линейный слой-релу-линейный

In [28]:
train_data_loader = torch.utils.data.DataLoader(
      train_dataset_resnet, batch_size = batch_size, shuffle = True
)
test_data_loader = torch.utils.data.DataLoader(
      test_dataset_resnet, batch_size = batch_size, shuffle = False
)

In [27]:
model = models.resnet18(models.ResNet18_Weights.DEFAULT)
for params in model.parameters():
  params.requires_grad = False

input_layers = model.fc.in_features
model.fc = torch.nn.Linear(input_layers, 10)
model.fc = torch.nn.Sequential(
        torch.nn.Linear(input_layers, input_layers//2),
        torch.nn.ReLU(),
        torch.nn.Linear(input_layers//2, 10),
        )
model.to(device)
optimizer = torch.optim.Adam(model.fc.parameters(), lr = learning_rate)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


Train

In [29]:
train(model, train_data_loader, device, optimizer)

Epoch 0: loss = 0.9640857577323914, acc = 0.6889605522155762, time = 176.2004740729999
Epoch 1: loss = 0.6718161106109619, acc = 0.7661934494972229, time = 171.12686706099976


Test

In [30]:
loss_list_test = []
acc_list_test = []
for batch_idx, (images, labels) in enumerate(test_data_loader):
    images, labels = images.to(device), labels.to(device)
    optimizer.zero_grad()
    outputs = model(images)
    loss = loss_function(outputs, labels)
    loss_list.append(loss)
    acc_list.append(current_accuracy(outputs, labels))
    loss.backward()
    optimizer.step()

print('Test loss = {}, Test acc = {}'.format(torch.stack(loss_list).mean(), torch.stack(acc_list).mean()))

Test loss = 2.043820381164551, Test acc = 0.21254104375839233


## ShuffleNet_V2_X1_5

In [32]:
train_dataset_shufflenet = torchvision.datasets.CIFAR10(
      root = dir_name, train = True, download = True,
      transform = models.ShuffleNet_V2_X1_5_Weights.DEFAULT.transforms()
)
test_dataset_shufflenet = torchvision.datasets.CIFAR10(
      root = dir_name, train = False, download = True,
      transform = models.ShuffleNet_V2_X1_5_Weights.DEFAULT.transforms()
)

Files already downloaded and verified
Files already downloaded and verified


In [33]:
train_data_loader = torch.utils.data.DataLoader(
      train_dataset_shufflenet, batch_size = batch_size, shuffle = True
)
test_data_loader = torch.utils.data.DataLoader(
      test_dataset_shufflenet, batch_size = batch_size, shuffle = False
)

Плюс один линейный слой

In [35]:
model = models.shufflenet_v2_x1_5(models.ShuffleNet_V2_X1_5_Weights.DEFAULT)
for params in model.parameters():
  params.requires_grad = False

input_layers = model.fc.in_features
model.fc = torch.nn.Sequential(
        torch.nn.Linear(input_layers, input_layers//2),
        torch.nn.ReLU(),
        torch.nn.Linear(input_layers//2, 10),
        )
model.to(device)
optimizer = torch.optim.Adam(model.fc.parameters(), lr = learning_rate)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
Downloading: "https://download.pytorch.org/models/shufflenetv2_x1_5-3c479a10.pth" to /root/.cache/torch/hub/checkpoints/shufflenetv2_x1_5-3c479a10.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 55.1MB/s]


Train

In [36]:
train(model, train_data_loader, device, optimizer)

Epoch 0: loss = 1.24503755569458, acc = 0.6684479713439941, time = 157.0673728640004
Epoch 1: loss = 0.7350917458534241, acc = 0.7790235280990601, time = 152.7255569400004


Test

In [37]:
loss_list_test = []
acc_list_test = []
for batch_idx, (images, labels) in enumerate(test_data_loader):
    images, labels = images.to(device), labels.to(device)
    optimizer.zero_grad()
    outputs = model(images)
    loss = loss_function(outputs, labels)
    loss_list.append(loss)
    acc_list.append(current_accuracy(outputs, labels))
    loss.backward()
    optimizer.step()

print('Test loss = {}, Test acc = {}'.format(torch.stack(loss_list).mean(), torch.stack(acc_list).mean()))

Test loss = 1.8444106578826904, Test acc = 0.2960549294948578


Плюс линейный слой-релу-линейный

In [38]:
train_data_loader = torch.utils.data.DataLoader(
      train_dataset_shufflenet, batch_size = batch_size, shuffle = True
)
test_data_loader = torch.utils.data.DataLoader(
      test_dataset_shufflenet, batch_size = batch_size, shuffle = False
)

In [39]:
model = models.shufflenet_v2_x1_5(models.ShuffleNet_V2_X1_5_Weights.DEFAULT)
for params in model.parameters():
  params.requires_grad = False

input_layers = model.fc.in_features
model.fc = torch.nn.Linear(input_layers, 10)
model.to(device)
optimizer = torch.optim.Adam(model.fc.parameters(), lr = learning_rate)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


Train

In [40]:
train(model, train_data_loader, device, optimizer)

Epoch 0: loss = 1.2410244941711426, acc = 0.6689121723175049, time = 152.84427278699968
Epoch 1: loss = 0.7323034405708313, acc = 0.7809140086174011, time = 153.34246092099966


Test

In [41]:
loss_list_test = []
acc_list_test = []
for batch_idx, (images, labels) in enumerate(test_data_loader):
    images, labels = images.to(device), labels.to(device)
    optimizer.zero_grad()
    outputs = model(images)
    loss = loss_function(outputs, labels)
    loss_list.append(loss)
    acc_list.append(current_accuracy(outputs, labels))
    loss.backward()
    optimizer.step()

print('Test loss = {}, Test acc = {}'.format(torch.stack(loss_list).mean(), torch.stack(acc_list).mean()))

Test loss = 1.6952788829803467, Test acc = 0.35858818888664246
